In [1]:

!pip install sktime
!pip install tslearn
import tslearn
#https://tslearn.readthedocs.io/en/stable/index.html
print(tslearn.__version__)

     |████████████████████████████████| 5.7MB 5.3MB/s 
     |████████████████████████████████| 9.5MB 37.1MB/s 
     |████████████████████████████████| 22.3MB 54.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 798kB 5.2MB/s 
0.5.0.5


Preliminaries

In [3]:
import numpy as np
from sktime.utils.data_io import load_from_tsfile_to_dataframe
from tslearn.utils import from_sktime_dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


Load multivariate time series/panel data

In [6]:

X_train, y_train = load_from_tsfile_to_dataframe('/content/drive/MyDrive/TRAIN.ts')
X_test, y_test = load_from_tsfile_to_dataframe('/content/drive/MyDrive/TEST.ts')
X_train_arr = from_sktime_dataset(X_train)
X_test_arr = from_sktime_dataset(X_test)
print(y_train)
print(y_test)

'''
X, y = load_basic_motions(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
'''
print(X_train_arr.shape, y_train.shape, X_test_arr.shape, y_test.shape)

['elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'human' 'human'
 'human' 'human' 'human' 'human']
['elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'human' 'human' 'human']
(29, 1445, 2) (29,) (12, 779, 2) (12,)


In [7]:
#with np.printoptions(threshold=np.inf):
  #print(X_train_arr)
  #print(np.argwhere(np.isnan(X_train_arr)))

UNEQUAL LENGTH MULTIVARIATE: KNN with Dynamic Time Warping (DTW) Similarity Metrics

In [8]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
knn = KNeighborsTimeSeriesClassifier(n_neighbors=1, metric='softdtw')
knn.fit(X_train_arr, y_train)
print(knn.predict(X_train_arr))
print(knn.predict(X_test_arr))
knn.score(X_test_arr, y_test)

['elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'elephant' 'elephant' 'elephant' 'human' 'human'
 'human' 'human' 'human' 'human']
['human' 'elephant' 'human' 'human' 'elephant' 'elephant' 'elephant'
 'elephant' 'elephant' 'human' 'human' 'human']


0.75

UNEQUAL LENGTH MULTIVARIATE: Support Vector Classifier

In [32]:
from tslearn.svm import TimeSeriesSVC
#reduce value of gamma for address NaN error
svc = TimeSeriesSVC(kernel="gak", gamma=0.2, C=1.0)
svc.fit(X_train_arr, y_train)
print(svc.predict(X_test_arr))
svc.score(X_test_arr, y_test)

0.75

UNEQUAL LENGTH MULTIVARIATE: Learning Shapelets Method

In [33]:
from tslearn.shapelets import LearningShapelets
shap = LearningShapelets(n_shapelets_per_size={3: 1})
shap.fit(X_train_arr, y_train)
print(shap.predict(X_test_arr))
shap.score(X_test_arr, y_test)

/usr/local/lib/python3.7/dist-packages/tslearn/shapelets/shapelets.py:357: FutureWarning: The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.
  FutureWarning)


0.75

#STOP

In [ ]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.datasets.base import load_japanese_vowels  # multivariate dataset
from sktime.transformations.panel.rocket import Rocket

In [ ]:
#X_train, y_train = load_japanese_vowels(split="train", return_X_y=True)

rocket = Rocket()
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)

ValueError: ignored

In [ ]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

In [ ]:
X_test, y_test = load_japanese_vowels(split="test", return_X_y=True)
X_test_transform = rocket.transform(X_test)

classifier.score(X_test_transform, y_test)

1.0

#DO NOT RUN THE METHODS BELOW AS THEY ARE FOR EQUAL LENGTH SERIES

EQUAL LENGTH MULTIVARIATE: 
Time series concatenation

In [ ]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100)),
]
clf = Pipeline(steps)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

EQUAL LENGTH MULTIVARIATE: 
Column ensembling

In [ ]:
clf = ColumnEnsembleClassifier(
    estimators=[
        ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
        ("BOSSEnsemble3", BOSSEnsemble(max_ensemble_size=5), [3]),
    ]
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

EQUAL LENGTH MULTIVARIATE: Bespoke classification algorithms


In [ ]:
clf = MrSEQLClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)